In [1]:
!nvidia-smi

Wed Jun  9 16:59:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 440.36       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
| 81%   78C    P2   194W / 280W |   2793MiB / 11175MiB |     71%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 37%   63C    P2   189W / 280W |   2377MiB / 11178MiB |     65%      Default |
+-------------------------------+----------------------+----------------------+
        

##### 计算设备
所有的深度学习框架
默认在cpu上做运算

In [2]:
import torch
from torch import nn

torch.device('cpu'),torch.cuda.device('cuda'),torch.cuda.device('cuda:1')
# 查看cpu设备、第0个gpu，第1个gpu

(device(type='cpu'),
 <torch.cuda.device at 0x7f1fbc22a278>)

#### 查询可用gpu的数量

In [3]:
torch.cuda.device_count()

2

#### 这两个函数允许我们在请求的GPU不存在的情况下运行代码

In [4]:
def try_gpu(i=0):  
    """如果存在，则返回gpu(i)，否则返回cpu()。"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0), device(type='cuda', index=1)])

#### 查询张量存在的设备

In [5]:
x = torch.tensor([1,2,3])
x.device

device(type='cpu')

#### 存储在gpu上

In [6]:
X = torch.ones(2,3,device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

#### 第二个gpu上创建一个随机张量

In [7]:
Y = torch.rand(2,3,device=try_gpu(1))
Y

tensor([[0.0920, 0.4711, 0.3697],
        [0.9173, 0.7190, 0.8875]], device='cuda:1')

#### 要计算X+Y，需要决定在那里执行这个操作
把放在不同gpu的张量移到一个gpu上执行

In [8]:
z = X.cuda(1)
print(X)
print(z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


#### 现在数据同在一个GPU上（Z和Y都在1号gpu），可以将它们相加

In [9]:
Y + z

tensor([[1.0920, 1.4711, 1.3697],
        [1.9173, 1.7190, 1.8875]], device='cuda:1')

In [10]:
z.cuda(1) is z

True

#### 神经网络与GPU

In [12]:
net = nn.Sequential(nn.Linear(3,1))
net = net.to(device=try_gpu())   # 移动到gpu上

net(X)

tensor([[-0.8964],
        [-0.8964]], device='cuda:0', grad_fn=<AddmmBackward>)

#### 确认模型参数存储在同一个gpu上

In [13]:
net[0].weight.data.device

device(type='cuda', index=0)